# OpenAI APIs Assignment: Prompt Engineering & Agent Pattern: Tool Use

## Assignment Overview
This assignment focuses on the following scenarios:
* Tool Use with Wikipedia: Implementing function calling with Wikipedia integration

## Assignment Structure: Tool Use with Wikipedia Integration 
Objective: Create a research assistant that can search Wikipedia and provide comprehensive information using OpenAI's function calling capabilities.

## Requirements:
* Wikipedia Search Function: Implement a function to search Wikipedia articles
* Content Extraction: Extract and summarize article content
* Function Calling: Use OpenAI's function calling to integrate Wikipedia data
* Structured Output: Return information in a structured format
* Error Handling: Handle cases where articles don't exist or are ambiguous

## Key Implementation Tasks:

```# Required libraries
import wikipedia
import openai
from dotenv import load_dotenv

# Function 1: Wikipedia Search
def search_wikipedia(query):
    """Search Wikipedia for articles related to the query"""
    # Students implement Wikipedia search using wikipediaapi
    # Return structured results with titles and descriptions

# Function 2: Content Extraction  
def get_wikipedia_content(title):
    """Get full content of a Wikipedia article"""
    # Students implement content extraction
    # Handle missing articles and disambiguation

# Function 3: OpenAI Function Calling
def handle_wikipedia_research(query):
    """Complete function calling workflow"""
    # Students implement the full workflow:
    # 1. Send query to OpenAI with function definitions
    # 2. Handle function calls (search_wikipedia, get_wikipedia_content)
    # 3. Execute requested functions
    # 4. Send results back to OpenAI for final response
```

## You can use one of these test cases to show:
* "What is artificial intelligence?"
* "Tell me about the history of Singapore"
* "Explain quantum computing"


## How to submit your assignment to Canvas
Submit the code with the input and output as a pdf file into Canvas 


In [1]:
#Here is an example of the Wikipedia Search Function and you may need to install the wikipedia library API
# import wikipedia

# def get_article(search_term):
#     results = wikipedia.search(search_term)
#     first_result = results[0]
#     page = wikipedia.page(first_result, auto_suggest=False)
#     return page.content

# article = get_article("What is artificial intelligence?")
# #print(article[:1000]) 
# #article is very long, so let's just print a preview

# from IPython.display import display, HTML

# article = get_article("What is artificial intelligence?")
# html_content = f"""
# <div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #007bff;">
#     <h3 style="color: #007bff; margin-top: 0;">What is artificial intelligence? Answer Preview</h3>
#     <p style="line-height: 1.6; text-align: justify;">{article[:1000]}...</p>
#     <small style="color: #6c757d;">(Showing first 1000 characters)</small>
# </div>
# """
# display(HTML(html_content))

I used GPT-4.0 to produce sample codes for creation of some functions which I modified to suit assginment requirements. I am responsible for the content and quality of the submitted work.

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # looks for .env in the current or parent directory




True

In [3]:
# Function 1
import requests

def search_wikipedia(query):
    """
    REST API function that returns title, and description 
    for the top 5 search results.
    
    Args:
        query (str): The query to search for on Wikipedia
        
    Returns:
        list: A list of dictionaries containing 'title' and'description' 
              for up to 5 Wikipedia articles
    """
    language_code = "en"
    base_url = f"https://api.wikimedia.org/core/v1/wikipedia/{language_code}/search/page"
    
    headers = {
        'User-Agent': 'MyApp/1.0 (your.email@example.com)',
    }
    
    params = {
        'q': query,
        'limit': 5
    }
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        if 'pages' not in data or not data['pages']:
            return []

        articles = []

        for page_data in data['pages']:
            title = page_data.get('title')
            raw_description = page_data.get('description')
            description = raw_description.strip() if raw_description and raw_description.strip() else 'No description available'


            article_info = {
                'title': title,
                'description': description
 
            }
            articles.append(article_info)


        return articles

    except Exception as e:
        print(f"Error querying Wikimedia REST API: {e}")
        return []



# Example usage and testing
if __name__ == "__main__":
    query = "What is artificial intelligence"
    print(f"Searching for: {query}")
    print("=" * 50)

    articles = search_wikipedia(query)

    for i, article in enumerate(articles, 1):
        print(f"\n--- Article {i} ---")
        print(f"Title: {article['title']}")
        print(f"Description: {article['description']}...")



Searching for: What is artificial intelligence

--- Article 1 ---
Title: Artificial intelligence
Description: Intelligence of machines...

--- Article 2 ---
Title: Artificial general intelligence
Description: Type of AI with wide-ranging abilities...

--- Article 3 ---
Title: Applications of artificial intelligence
Description: No description available...

--- Article 4 ---
Title: Friendly artificial intelligence
Description: AI to benefit humanity...

--- Article 5 ---
Title: Hallucination (artificial intelligence)
Description: Erroneous material generated by AI...


In [4]:
# helper function
def process_articles(articles):
    """
    Process a list of Wikipedia articles and return their titles.

    Args:
        articles (list): A list of dictionaries containing article information.

    Returns:
        list: A list containing titles of articles
    """
    titles = []
    for article in articles:
        titles.append(article.get('title'))
    return titles


In [5]:
# 2nd function
import wikipedia

def get_wikipedia_content(title):
    """
    Using Wikipedia python library returns contents 
    for the top 3 search results.
    
    Args:
        title (str): The title of the Wikipedia article to retrieve content for.

    Returns:
        content (str): The content of the Wikipedia article
    """
    

    try:
        # Use wikipedia package to get full content
        page = wikipedia.page(title, auto_suggest=False)

        content = page.content

    except wikipedia.exceptions.DisambiguationError as e:
        # Try first disambiguation option
        try:
            page = wikipedia.page(e.options[0], auto_suggest=False)
            content = page.content
        except:
            print(f"Disambiguation error occurred for '{title}'")
            return []
    except wikipedia.exceptions.PageError:
        print(f"Missing article occurred for '{title}'")
        return []
    except Exception as e:
        print(f"Error retrieving Wikipedia content for '{title}': {e}")
        return []
        
    return content


# Example usage and testing
if __name__ == "__main__":
    title = "Artificial Intelligence"
    print(f"Extracting content for: {title}")
    print("=" * 50)

    content = get_wikipedia_content(title)

    print(f"Content for '{title}':\n{content[:500]}...")  # Print first 500 characters of content
  


ModuleNotFoundError: No module named 'wikipedia'

In [ ]:
from agents import Agent, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from IPython.display import Markdown
import json
from IPython.display import display, HTML

# 1. Define your tool (Function 3 orchestrator)
@function_tool
def handle_wikipedia_research(query: str):
    """
    Handles the Wikipedia research process for a given query.
    Uses search_wikipedia() + get_wikipedia_content() internally.
    """
    articles = search_wikipedia(query)          # Function 1
    titles = process_articles(articles)         # Helper

    contents = []
    for title in titles:
        content = get_wikipedia_content(title)  # Function 2
        contents.append(content)

    return json.dumps(contents)


# 2. Instructions for the agent
INSTRUCTIONS = (
    "You are a research assistant. You are to use the extracted information from Wikipedia "
    "to answer user queries. Your goal is to provide accurate and concise answers based on "
    "the retrieved content from Wikipedia. If you do not know the answer, please say so."
)

# 3. Create the Agent object
research_agent = Agent(
    name="Research agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
    tools=[handle_wikipedia_research],  # ✅ pass the wrapped tool
)

# 4. Run the agent
message = (
    "You are a research assistant. You have to provide a simple explanation of neural networks. "
    "The goal is to help users understand complex topics easily. The target audience is elementary school students."
)

with trace("Research_Agent_Search"):
    result = await Runner.run(research_agent, message)

# 5. Display the answer
display(Markdown(result.final_output))

# final=result.final_output

# html_content = f"""
# <div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border-left: 4px solid #007bff;">
#     <h3 style="color: #007bff; margin-top: 0;">What is artificial intelligence? Answer Preview</h3>
#     <p style="line-height: 1.6; text-align: justify;color: #6c757d;">{final}...</p>
#     <small style="color: #6c757d;">(Showing first 1000 characters)</small>
# </div>
# """
# display(HTML(html_content))


### What Are Neural Networks?

**Neural networks** are like smart computer brains. They help computers learn from information so that they can make decisions or predictions, almost like how we learn from our experiences.

#### How Do They Work?

1. **Tiny Neurons**: Imagine little pieces of a brain called **neurons**. Each neuron connects to many others, just like friends passing notes in class.
  
2. **Layers**: Neurons are grouped into **layers**. There’s an **input layer** (where information comes in), hidden layers (where the computer figures things out), and an **output layer** (where the answer comes out).

3. **Learning through Examples**: Just like you learn by practicing, neural networks learn by seeing lots of examples. They adjust their connections (like changing how you pass notes) to improve their guesses.

4. **Making Predictions**: Once trained, they can guess things like whether a picture is of a cat or a dog!

### Why Are They Important?

Neural networks are used in many everyday things, like helping robots understand speech, recognizing faces in photos, and suggesting videos online. They help computers work in smarter ways, making them useful in tons of different jobs!

### In Summary

Neural networks are computer systems that learn from examples to make smart decisions, using a structure inspired by how our brains work. They can help solve many problems and make our lives easier!